# Problem set 2: Finding the Walras equilibrium in a multi-agent economy

[<img src="https://mybinder.org/badge_logo.svg">](https://mybinder.org/v2/gh/NumEconCopenhagen/exercises-2020/master?urlpath=lab/tree/PS2/problem_set_2.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2

# Tasks

## Drawing random numbers

Replace the missing lines in the code below to get the same output as in the answer.

In [2]:
import numpy as np
np.random.seed(1986)
state = np.random.get_state()
for i in range(3):
    np.random.set_state(state)
    for j in range(2):
        x = np.random.uniform()
        print(f'({i},{j}): x = {x:.3f}')

(0,0): x = 0.569
(0,1): x = 0.077
(1,0): x = 0.569
(1,1): x = 0.077
(2,0): x = 0.569
(2,1): x = 0.077


**Answer:**

In [3]:
import numpy as np
np.random.seed(1986)
state = np.random.get_state()
for i in range(3):
    np.random.set_state(state)
    for j in range(2):
        x = np.random.uniform()
        print(f'({i},{j}): x = {x:.3f}')

(0,0): x = 0.569
(0,1): x = 0.077
(1,0): x = 0.569
(1,1): x = 0.077
(2,0): x = 0.569
(2,1): x = 0.077


## Find the expectated value

Find the expected value and the expected variance

$$ 
\mathbb{E}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} g(x_i
$$
$$ 
\mathbb{VAR}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} \left( g(x_i) - \frac{1}{N}\sum_{i=1}^{N} g(x_i) \right)^2
$$

where $ x_i \sim \mathcal{N}(0,\sigma) $ and

$$ 
g(x,\omega)=\begin{cases}
x & \text{if }x\in[-\omega,\omega]\\
-\omega & \text{if }x<-\omega\\
\omega & \text{if }x>\omega
\end{cases} 
$$

In [4]:
sigma = 3.14
omega = 2
N = 10000
np.random.seed(1986)

x_vec = np.random.normal(0,sigma,N)
def g(x_vec,omega=omega):
    return np.fmin(omega,np.fmax(-omega,x_vec))

g_vec = g(x)

print(f'E(g(x)):   {np.mean(g_vec)}')
print(f'Var(g(x)): {np.var(g_vec)}')

## self defined functions: 
def mean_fun(vec):
    return sum(vec)/len(vec)

def var_fun(vec):
    n = len(vec)
    mean = mean_fun(vec)
    return (1/n)*sum([(g- mean)**2 for g in vec])

print(f'E(g(x)):   {mean_fun(g_vec)}')
print(f'Var(g(x)): {var_fun(g_vec)}')

E(g(x)):   0.07733828848384083
Var(g(x)): 0.0


TypeError: 'numpy.float64' object is not iterable

**Answer:**

In [ ]:
# a. parameter choices
sigma = 3.14
omega = 2
N = 10000
np.random.seed(1986)

# b. draw random numbers
x = np.random.normal(loc=0,scale=sigma,size=N)

# c. transformation function
def g(x,omega):
    y = x.copy()
    y[x < -omega] = -omega
    y[x > omega] = omega
    return y

# d. mean and variance
mean = np.mean(g(x,omega))
var = np.var(g(x-mean,omega))
print(f'mean = {mean:.5f}, var = {var:.5f}')

## Interactive histogram

**First task:** Consider the code below. Fill in the missing lines so the figure is plotted.

In [ ]:
# a. import
%matplotlib inline
import matplotlib.pyplot as plt
# missing line
from scipy.stats import norm

# b. plotting figure
def fitting_normal(X,mu_guess,sigma_guess):
    
    # i. normal distribution from guess
    F = norm(loc=mu_guess,scale=sigma_guess)
    
    # ii. x-values
    x_low = F.ppf(0.001)
    x_high =F.ppf(0.999)
    x = np.linspace(x_low,x_high,100)

    # iii. figure
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    ax.plot(x,F.pdf(x),lw=2)
    ax.hist(X,bins=100,density=True,histtype='stepfilled');
    ax.set_ylim([0,0.5])
    ax.set_xlim([-6,6])

# c. parameters
mu_true = 2
sigma_true = 1
mu_guess = 1
sigma_guess = 2

# d. random draws
X = np.random.normal(loc=mu_true,scale=sigma_true,size=10**6)

# e. figure
try:
    fitting_normal(X,mu_guess,sigma_guess)
except:
    print('failed')

**Second task:** Create an interactive version of the figure with sliders for $\mu$ and $\sigma$.

In [5]:
widgets.interact(fitting_normal,
    mu_guess=widgets.FloatSlider(description="$\mu$", min=0.1, max=5, step=0.05, value=1),
    sigma_guess=widgets.FloatSlider(description="$\sigma$", min=0.1, max=5, step=0.05, value=1),
    X=widgets.fixed(X)
);

NameError: name 'widgets' is not defined

**Answer:**

In [ ]:
# a. import
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm

# b. plotting figure
def fitting_normal(X,mu_guess,sigma_guess):
    
    # i. normal distribution from guess
    F = norm(loc=mu_guess,scale=sigma_guess)
    
    # ii. x-values
    x_low = F.ppf(0.001) # x value where cdf is 0.001
    x_high = F.ppf(0.999) # x value where cdf is 0.999
    x = np.linspace(x_low,x_high,100)

    # iii. figure
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    ax.plot(x,F.pdf(x),lw=2)
    ax.hist(X,bins=100,density=True,histtype='stepfilled');
    ax.set_ylim([0,0.5])
    ax.set_xlim([-6,6])

# c. parameters
mu_true = 2
sigma_true = 1
mu_guess = 1
sigma_guess = 2

# d. figure
X = np.random.normal(loc=mu_true,scale=sigma_true,size=10**6)
#fitting_normal(X,mu_guess,sigma_guess)

In [ ]:
import ipywidgets as widgets
widgets.interact(fitting_normal,
    X=widgets.fixed(X),
    mu_guess=widgets.FloatSlider(description="$\mu$", min=0.1, max=5, step=0.05, value=1),
    sigma_guess=widgets.FloatSlider(description="$\sigma$", min=0.1, max=5, step=0.05, value=1)
);

## Save and load

Consider the code below and fill in the missing lines so the code can run without any errors.

In [6]:
import pickle

# a. create some data
my_data = {}
my_data['A'] = {'a':1,'b':2}
my_data['B'] = np.array([1,2,3])
my_data['C'] = np.array([20,25])


my_np_data = {}
my_np_data['D'] = np.array([1,2,3])
my_np_data['E'] = np.zeros((5,8))
my_np_data['F'] = np.linspace(0,1,10)

# c. save with pickle
with open(f'data.p', 'wb') as f:
    pickle.dump(my_data,f)
    
    
# d. save with numpy
np.savez(f'data.npz', **my_np_data)
    
# a. try
def load_all():
    with open(f'data.p', 'rb') as f:
        data = pickle.load(f)
        A = data['A']
        B = data['B']
        C = data['C']

    with np.load(f'data.npz') as data:
        D = data['D']
        E = data['E']
        F = data['F']        
    
    print('variables loaded without error')
    
try:
    load_all()
except:
    print('failed')

variables loaded without error


**Answer:**

In [7]:
import pickle

# a. create some data
my_data = {}
my_data['A'] = {'a':1,'b':2}
my_data['B'] = np.array([1,2,3])
my_data['C'] = (1,4,2)

my_np_data = {}
my_np_data['D'] = np.array([1,2,3])
my_np_data['E'] = np.zeros((5,8))
my_np_data['F'] = np.ones((7,3,8))

# c. save with pickle
with open(f'data.p', 'wb') as f:
    pickle.dump(my_data, f)
    
# d. save with numpy
np.savez(f'data.npz', **my_np_data)
    
# a. try
def load_and_print():
    with open(f'data.p', 'rb') as f:
        data = pickle.load(f)
        A = data['A']
        B = data['B']
        C = data['C']

    with np.load(f'data.npz') as data:
        D = data['D']
        E = data['E']
        F = data['F']        

    print('variables loaded without error')
            
try:
    load_and_print()
except:
    print('an error is found')

variables loaded without error


## Modules

1. Call the function `myfun` from the module `mymodule` present in this folder.
2. Open VSCode and open the `mymodule.py`, add a new function and call it from this notebook.

In [8]:
import mymodule
mymodule.myfun(5)

hello world!
hello world!
hello world!
hello world!
hello world!


In [9]:
mymodule.newfun('hey')

heyheyheyheyhey


**Answer:**

In [10]:
import mymodule
mymodule.myfun(5)

hello world!
hello world!
hello world!
hello world!
hello world!


## Git

1. Follow this [guide](https://numeconcopenhagen.netlify.com/guides/vscode-git/) to create a GitHub repository, and put your solution to this problem set in it.
2. Pair up with a fellow student. Clone each others repositories and run the code in them.

**IMPORTANT:** You will need **git** for the data project in a few needs. Better learn it know. Remember, that the teaching assistants are there to help you.

# Problem

Consider an **exchange economy** with

1. 2 goods, $(x_1,x_2)$
2. $N$ consumers indexed by $j \in \{1,2,\dots,N\}$
3. Preferences are Cobb-Douglas with truncated normally *heterogenous* coefficients

    $$
    \begin{aligned}
    u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha_{j}}x_{2}^{1-\alpha_{j}}\\
     & \tilde{\alpha}_{j}\sim\mathcal{N}(\mu,\sigma)\\
     & \alpha_j = \max(\underline{\mu},\min(\overline{\mu},\tilde{\alpha}_{j}))
    \end{aligned}
    $$

4. Endowments are *heterogenous* and given by

    $$
    \begin{aligned}
    \boldsymbol{e}^{j}&=(e_{1}^{j},e_{2}^{j}) \\
     &  & e_i^j \sim f, f(x,\beta_i) =  1/\beta_i \exp(-x/\beta)
    \end{aligned}
    $$

**Problem:** Write a function to solve for the equilibrium.

You can use the following parameters:

In [16]:
# a. parameters
N = 10000
mu = 0.5
sigma = 0.2
mu_low = 0.1
mu_high = 0.9
beta1 = 1.3
beta2 = 2.1
seed = 1986

# b. draws of random numbers

e1_vec = np.random.exponential(beta1,size=N)
e2_vec= np.random.exponential(beta2,size=N)
alpha_raw_vec = np.random.normal(mu,sigma,size=N)

alpha_vec = np.fmin(mu_high,np.fmax(mu_low,alpha_raw_vec))

# c. demand function
def demand_good1(e1,e2,alpha,p):
    I = e1*p+e2
    return alpha*I/p

# d. excess demand function

# Precalculated as it is constant
supply_good1 = np.sum(e1_vec)

def excess_demand_good1(e1_vec,e2_vec,alpha_vec,p1,supply_good1):
    total_demand = np.sum(demand_good1(e1_vec,e2_vec,alpha_vec,p1))
    return total_demand-supply_good1


# e. find equilibrium function
def find_equilibrium(e1_vec,e2_vec,alpha_vec,p1_guess,supply_good1,kappa=0.5,eps=1e-8,maxiter=500):
    
    # Guess 
    p1 = p1_guess
    t = 0
    while True:

        # a. step 1: excess demand
        Z1 = excess_demand_good1(e1_vec,e2_vec,alpha_vec,p1,supply_good1)
        
        # b: step 2: stop?
        if  np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
            break    
    
        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alpha_vec.size
            
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p1

# f. call find equilibrium function
p1_optimal = find_equilibrium(e1_vec,e2_vec,alpha_vec,1,supply_good1)

  0: p1 =   1.20217783 -> excess demand ->  4043.55650344
  1: p1 =   1.31553991 -> excess demand ->  2267.24161898
  2: p1 =   1.39104725 -> excess demand ->  1510.14689928
  3: p1 =   1.44476403 -> excess demand ->  1074.33564828
  4: p1 =   1.48436531 -> excess demand ->   792.02560130
   ...
 25: p1 =   1.61920941 -> excess demand ->     5.52176723
 50: p1 =   1.62030069 -> excess demand ->     0.02008215
 75: p1 =   1.62030466 -> excess demand ->     0.00007318
100: p1 =   1.62030467 -> excess demand ->     0.00000027
115: p1 =   1.62030467 -> excess demand ->     0.00000001


**Hint:** The code structure is exactly the same as for the exchange economy considered in the lecture. The code for solving that exchange economy is reproduced in condensed form below.

In [15]:
# a. parameters
N = 1000
k = 2
mu_low = 0.1
mu_high = 0.9
seed = 1986

# b. draws of random numbers
np.random.seed(seed)
alphas = np.random.uniform(low=mu_low,high=mu_high,size=N)

# c. demand function
def demand_good_1_func(alpha,p1,p2,k):
    I = k*p1+p2
    return alpha*I/p1

# d. excess demand function
def excess_demand_good_1_func(alphas,p1,p2,k):
    
    # a. demand
    demand = np.sum(demand_good_1_func(alphas,p1,p2,k))
    
    # b. supply
    supply = k*alphas.size
    
    # c. excess demand
    excess_demand = demand-supply
    
    return excess_demand

# e. find equilibrium function
def find_equilibrium(alphas,p1,p2,k,kappa=0.5,eps=1e-8,maxiter=500):
    
    t = 0
    while True:

        # a. step 1: excess demand
        Z1 = excess_demand_good_1_func(alphas,p1,p2,k)
        
        # b: step 2: stop?
        if  np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
            break    
    
        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alphas.size
            
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p1

# e. call find equilibrium function
p1 = 1.4
p2 = 1
kappa = 0.1
eps = 1e-8
p1 = find_equilibrium(alphas,p1,p2,k,kappa=kappa,eps=eps)

  0: p1 =   1.33690689 -> excess demand ->  -630.93108302
  1: p1 =   1.27551407 -> excess demand ->  -613.92820358
  2: p1 =   1.21593719 -> excess demand ->  -595.76882769
  3: p1 =   1.15829785 -> excess demand ->  -576.39340748
  4: p1 =   1.10272273 -> excess demand ->  -555.75114178
   ...
 25: p1 =   0.53269252 -> excess demand ->   -53.80455643
 50: p1 =   0.50897770 -> excess demand ->    -0.27125769
 75: p1 =   0.50886603 -> excess demand ->    -0.00120613
100: p1 =   0.50886553 -> excess demand ->    -0.00000536
125: p1 =   0.50886553 -> excess demand ->    -0.00000002
130: p1 =   0.50886553 -> excess demand ->    -0.00000001


**Answers:**

In [16]:
# a. parameters
N = 10000
mu = 0.5
sigma = 0.2
mu_low = 0.1
mu_high = 0.9
beta1 = 1.3
beta2 = 2.1
seed = 1986

# b. draws of random numbers
np.random.seed(seed)
alphas = np.random.normal(loc=mu,scale=sigma,size=N)
alphas = np.fmax(np.fmin(alphas,mu_high),mu_low)
e1 = np.random.exponential(beta1,size=N)
e2 = np.random.exponential(beta2,size=N)

# c. demand function
def demand_good_1_func(alpha,p1,p2,e1,e2):
    I = p1*e1+p2*e2
    return alpha*I/p1

# d. excess demand function
def excess_demand_good_1_func(alphas,p1,p2,e1,e2):
    
    # a. demand
    demand = np.sum(demand_good_1_func(alphas,p1,p2,e1,e2))
    
    # b. supply
    supply = np.sum(e1)
    
    # c. excess demand
    excess_demand = demand-supply
    
    return excess_demand

# e. find equilibrium function
def find_equilibrium(alphas,p1,p2,e1,e2,kappa=0.5,eps=1e-8,maxiter=500):
    
    t = 0
    while True:

        # a. step 1: excess demand
        Z1 = excess_demand_good_1_func(alphas,p1,p2,e1,e2)
        
        # b: step 2: stop?
        if  np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
            break    
    
        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alphas.size
            
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p1

# f. call find equilibrium function
p1 = 1.4
p2 = 1
kappa = 0.5
eps = 1e-8
p1 = find_equilibrium(alphas,p1,p2,e1,e2,kappa=kappa,eps=eps)

  0: p1 =   1.45140633 -> excess demand ->  1028.12662815
  1: p1 =   1.48943510 -> excess demand ->   760.57530384
  2: p1 =   1.51816180 -> excess demand ->   574.53408777
  3: p1 =   1.54017076 -> excess demand ->   440.17912761
  4: p1 =   1.55720246 -> excess demand ->   340.63398830
   ...
 25: p1 =   1.62002594 -> excess demand ->     2.71044780
 50: p1 =   1.62056127 -> excess demand ->     0.00980814
 75: p1 =   1.62056320 -> excess demand ->     0.00003553
100: p1 =   1.62056321 -> excess demand ->     0.00000013
112: p1 =   1.62056321 -> excess demand ->     0.00000001


# Extra Problems

## Multiple goods

Solve the main problem extended with multiple goods:

$$
\begin{aligned}
u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha^1_{j}} \cdot x_{2}^{\alpha^2_{j}} \cdots x_{M}^{\alpha^M_{j}}\\
 &  \alpha_j = [\alpha^1_{j},\alpha^2_{j},\dots,\alpha^M_{j}] \\
 &  \log(\alpha_j) \sim \mathcal{N}(0,\Sigma) \\
\end{aligned}
$$

where $\Sigma$ is a valid covariance matrix.

In [285]:
# a. choose parameters
N = 10000
J = 3
p_guess = np.ones(3)
# b. choose Sigma
Sigma_lower = np.array([[1, 0, 0], [0.5, 1, 0], [0.25, -0.5, 1]])
Sigma_upper = Sigma_lower.T
Sigma = Sigma_upper@Sigma_lower
print(Sigma)

# c. draw random numbers
np.random.seed(seed)
alphas = np.exp(np.random.multivariate_normal(np.zeros(J), Sigma, N))
print(np.mean(alphas,axis=0))
print(np.corrcoef(alphas.T))


# b. draws of random numbers
beta_vec = np.random.uniform(1,3,size=J)
e_matrix = np.random.exponential(beta_vec.T,size=(N,1,J))[:,0,:]


# c. demand function
def demand_goods(e_matrix,alphas,p_vec):
    I = e_matrix@p_vec
    return (alphas/np.sum(alphas,axis=1)[:,np.newaxis])* (I[:,np.newaxis]/p_vec)


supply_vec = np.sum(e_matrix,axis=0)

def excess_demands(e_matrix,alphas,p_vec,supply_vec=supply_vec):
    
    total_demand = np.sum(demand_goods(e_matrix,alphas,p_vec),axis=0)
    return total_demand-supply_vec



[[ 1.312  0.375  0.25 ]
 [ 0.375  1.25  -0.5  ]
 [ 0.25  -0.5    1.   ]]
[1.913 1.917 1.636]
[[ 1.     0.199  0.152]
 [ 0.199  1.    -0.161]
 [ 0.152 -0.161  1.   ]]


In [287]:
np.set_printoptions(precision=3)


# e. find equilibrium function
def find_equilibrium(e_matrix,alphas,p_guess,kappa=0.5,eps=1e-8,maxiter=500):
    
    # Guess 
    p = p_guess
    t = 0
    
    # Calculate supply
    supply_vec = np.sum(e_matrix,axis=0)
    
    while True:

        # a. step 1: excess demand
        Z = excess_demands(e_matrix,alphas,p)
        
        # b: step 2: stop?
        if  np.all(np.abs(Z[:-1]) < eps) or t >= maxiter:
            
            print(f'{t:3d}: p = {p} -> excess demand -> {Z}')
            break    
    
        # c. step 3: update p, by adjusting for excess demand for each good, pus adjusting for the last good
        p = p + kappa*Z/alphas[:,0].size - kappa*Z[-1]/alphas.size
        
        #Normalize list p to 1
        p[-1]=1
        
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p = {p} -> excess demand -> {Z}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p

# f. call find equilibrium function
p_optimal = find_equilibrium(e_matrix,alphas,p_guess)

  0: p = [0.932 1.162 1.   ] -> excess demand -> [-1742.697  2863.365 -1120.668]
  1: p = [0.95  1.187 1.   ] -> excess demand -> [ 190.037  323.664 -553.075]
  2: p = [0.957 1.2   1.   ] -> excess demand -> [  58.052  167.784 -254.366]
  3: p = [0.961 1.206 1.   ] -> excess demand -> [  26.812   79.667 -121.26 ]
  4: p = [0.962 1.209 1.   ] -> excess demand -> [ 12.632  38.213 -58.217]
   ...
 25: p = [0.964 1.211 1.   ] -> excess demand -> [ 2.880e-06  8.871e-06 -1.352e-05]
 35: p = [0.964 1.211 1.   ] -> excess demand -> [ 2.019e-09  6.130e-09 -9.408e-09]
